# Mobile Robotics

Kevin J. Walchko
created 19 Oct 2017

----

Now we are going to start talking about mobile robotics (robots that move) as apposed to the robot arm we used in the beginning. This will have some of the same ideas and you should recognize some familar things like rotation matricies and coordinate systems.

## References

- Siegwart, R., Nourbakhsh, I., *Introduction to Autonomous Mobile Robots*

In [1]:
%matplotlib inline 

import numpy as np # matrix manipulations
from matplotlib import pyplot as plt

# Notation

When we derive the equations of motion (EoM) for a robot, we will notation like the following:

| | |
|-----------|--------------|
| $x$       | position     |
| $\dot x$  | velocity     |
| $\ddot x$ | acceleration |

where $x$ is often referred to as a state variable, although it could be any letter of the alphabet or a Greek mathematical symbol. If we take the time derivative of that variable with respect to time, we get velocity. If we take the derivative a second time, we get acceleration. Throughout, we will try to differentiate between a coordinate systems' x and y values and a robot's state variable.

# Coordinate Systems

It is important for robots to be able to travel from one place to another. In order to determine where the robot is, we typically asign 2 reference frames. A global (e.g. inertial frame) so we can understand the dynamics of the robot and a body frame to understand what the orientation of the robot is relative to the global frame. Also, remember in a previous lesson, we talked about **pose**, which is the position and orientation of something relative to aother frame.

<img src="pics/robot-frame.png" width="400px">

Another things to note is how the body frame is oriented. There is no standard way *generally*, however, most systems use the x-direciton of the body axis as forward. Then for aerospace tyically the y-axis goes out the right wing and z-axis points down. Others dislike the z-axis pointing down, so they orient the y-axis out the leftside of the body and now the z-axis points up. There is not advantage or disadvantage with these, just personal preference.

<img src="pics/body.png" width="400px">

For our work with the roomba, we the only orientation we are concerned about is *yaw* (about the z-axis) which is typically associated with a cardinal (or compass) direction. We will represent the robot's pose (or also known as *state*) as (see Siegwart, eqns 3.1 - 3.2):

$$
\xi =
\begin{bmatrix}
  x \\
  y \\
  \theta
\end{bmatrix} \\
R^I_B(\theta) =
\begin{bmatrix}
  \cos(\theta) & \sin(\theta) & 0 \\
  \sin(\theta) & \cos(\theta) & 0 \\
  0 & 0 & 1
\end{bmatrix} \\
\xi_I = R^I_B(\theta) \xi_B \\
R^B_I(\theta) = inv(R^I_B(\theta)) = R^I_B(\theta)^T
$$

where subscript I refers to the inertial frame, subscript B refers to the body frame. Hopefully you also recognize this rotation matrix as a rotation about the z-axis by $\theta$. The speed of a wheel is a function of its wheel radius ($r$) and how fast the motor turns ($\dot \phi$).

$$
v_{wheel} = \dot \phi r \\
\dot \xi_I = R^I_B(\theta) \begin{bmatrix}
  \frac{r \dot \phi_1}{2} + \frac{r \dot \phi_2}{2} \\
  0 \\
  \frac{r \dot \phi_1}{2} - \frac{r \dot \phi_2}{2}
\end{bmatrix}
$$

For example, substituting the following values into the previous equations gives:

- $\theta = \frac{\pi}{2}$
- r = 1
- l = 1
- $\dot \phi_1 = 4$
- $\dot \phi_2 = 2$

$$
\dot \xi_I = \begin{bmatrix}
  \dot x \\
  \dot y \\
  \dot \theta
\end{bmatrix} = 
\begin{bmatrix}
  0 & -1 & 0 \\
  1 & 0 & 0 \\
  0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
  3 \\
  0 \\
  1
\end{bmatrix} = 
\begin{bmatrix}
  0 \\
  3 \\
  1
\end{bmatrix}
$$

# Robot Types

## Non-holonomic

- Contraints limit the possible movements
- This is typically a velocity contraint.
    - The robot can occupy any point in a plane at any orientation, but it may not be able to directly transition from one point/orientation to another
    - Cars and parallel parking are a great example of this

## Holonomic

- There are no contraints on your robot and sideways movement is possible

# Odometry

A variety of robots are able to determine the distance ($d$) travelled because they have encoders on their wheels (our roomba does) to track the angle the wheel has moved ($\phi$). This is similar to how cars all come with odometers to show the miles travelled. 

$$
d = r \phi
$$

Now the equation above should look familar. If the wheel turns all the way around ($2\pi$ radians), then that equation becomes the circumference of a circle. If the world was perfect (spoiler, it isn't), then you would get this:

![](pics/ideal-odometry.png)

By tracking how far each wheel moves, we should be able to determine if a robot is going straight forward, turning to the left in reverse, or whatever. Unfortunately, wheels slip, expecially when it turns, and the distance travelled becomes corrupted. This leads to errors in the true angle of the turn and leads to the following:

![](pics/real-odometry.png)

This, odometry by itself is often not useful. Even worse, tracking a robot's position by how its wheels perform is not reliable at best. This is one reason why GPS is so useful with robots.

![](pics/rover.jpg)

The Mars rover uses video odometry to calculate its pose. Stereo cameras identify common features to track (like we did earlier in the vision block) in both the left and right camera. Then computed each feature's 3d position and tracked it from frame to frame to determine the distance and velocity travelled. With proper camera systems, lots of calibration and math, these types of systems are very good.

# Control

We want a control system that drives the error ($e$), which is the difference between the goal pose and the current pose, to zero over time. The trick will be finding this gain K. Also note, K will usually not be a scalar value, but a matrix also.

$$
\begin{bmatrix}
  v \\
  \omega
\end{bmatrix}
= K e =
K \begin{bmatrix}
  x \\
  y \\
  \theta
\end{bmatrix}^B \\
\lim_{t\to\infty} e(t) = 0
$$

Again, the kinematic equations for a differential drive robot (like the roomba) is:

$$
\begin{bmatrix}
  \dot x \\
  \dot y \\
  \dot \omega
\end{bmatrix}^I =
\begin{bmatrix}
  \cos(\theta) & 0 \\
  \sin(\theta) & 0 \\
  0 & 1
\end{bmatrix}
\begin{bmatrix}
  v \\
  \omega
\end{bmatrix}^B
$$